In [1]:
import pandas as pd
import numpy as np
import pickle as p

In [2]:
data_all = pd.read_csv('data_all.csv')

# Test with Subset of Data
np.random.seed(41)
item_subset = list(np.random.choice(data_all['Item_ID'].unique(), size=10, replace=False))
data_subset = data_all.loc[data_all['Item_ID'].isin(item_subset)].copy()

# # Test with All Data
# data_subset = data_all.copy()

In [3]:
item_subset

[31, 2259, 1203, 5073, 102, 5501, 3818, 4270, 3767, 197]

### `/train`

In [4]:
import requests
import json

url = 'http://localhost:5000/train/'

data = data_subset.to_json()

payload = {'data': data,
           'cv_acc': True,
           'project_id': 1
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [5]:
# Model Training Result
r.json()['result']

'Success'

In [6]:
# CV Accuracies Returned
cv_accuracies = pd.DataFrame.from_dict(json.loads(r.json()['cv_acc']))
cv_accuracies

,item_id,avg_sales,r2_score,mae_score,mpe_score,rmse_score
0,31.0,45.218983,0.746516,10.983628,0.242899,14.902278
1,102.0,176.608917,0.653418,39.211079,0.222022,54.829053
2,197.0,55.278580,0.040430,19.359501,0.350217,25.050150
3,1203.0,198.355331,0.749924,32.322582,0.162953,44.707349
4,2259.0,97.155861,0.636758,21.877142,0.225176,28.609255
5,3767.0,286.172363,0.763757,44.527010,0.155595,61.168836
6,3818.0,78.448463,0.569775,18.185342,0.231813,24.183214
7,4270.0,45.090721,-0.739857,27.625597,0.612667,34.656809
8,5073.0,115.369820,0.392978,39.100641,0.338916,51.085429
9,5501.0,464.741608,0.547121,122.214540,0.262973,157.871748


### `/optimize`

In [14]:
url = 'http://localhost:5000/optimize/'

payload = {'project_id': 1,
           'constraints': [],
           'population': 100,
           'max_epoch': 200
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [15]:
best_prices = r.json()['result']
best_prices

{'102': 4.36,
 '1203': 12.41,
 '197': 10.97,
 '2259': 4.3,
 '31': 14.36,
 '3767': 9.92,
 '3818': 10.82,
 '4270': 8.87,
 '5073': 4.02,
 '5501': 5.2}

In [16]:
import requests
import json

url = 'http://localhost:5000/predict/'
payload = {'prices': best_prices,
           'project_id': 1
           }
headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }
payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

# What If Prediction
qty = r.json()['qty_estimates']

In [17]:
qty

{'Qty_102': 450,
 'Qty_1203': 627,
 'Qty_197': 117,
 'Qty_2259': 229,
 'Qty_31': 142,
 'Qty_3767': 879,
 'Qty_3818': 223,
 'Qty_4270': 104,
 'Qty_5073': 270,
 'Qty_5501': 954}

In [18]:
# Calculate Revenue
revenue = 0
for item in best_prices.keys():
    revenue += (best_prices[item]*qty['Qty_'+item])
    
print(revenue)

32151.600000000002


### `/predict/`

In [19]:
# Sample Actual Price 
from helper_functions import *

sales_data = data_subset

sales_data = optimize_memory(sales_data)

sales_data_wide = sales_data.set_index(
    ['Wk', 'Tier', 'Store', 'Item_ID']).unstack(level=-1).reset_index().copy()
sales_data_wide.columns = [
    ''.join(str(i) for i in col).strip()
    for col in sales_data_wide.columns.values
]
sales_data_wide = sales_data_wide.sort_values(
    ['Tier', 'Store', 'Wk'], ascending=True).reset_index(drop=True)

sales_data_wide_clean = sales_data_wide.dropna(axis=0).copy()
dataset = sales_data_wide_clean
price_columns = [
    col for col in sales_data_wide_clean.columns if col.startswith('Price')
]

a = dataset.sample(1)
a_dict = a[price_columns].iloc[0].to_dict()
a_input = {}
for key in a_dict.keys():
    a_input[key.split('_')[1]]=round(a_dict[key],2)

print('Input Prices Quantities:')
a_input

Input Prices Quantities:


{'31': 2.0,
 '102': 9.45,
 '197': 10.4,
 '1203': 3.2,
 '2259': 12.95,
 '3767': 10.45,
 '3818': 10.3,
 '4270': 8.3,
 '5073': 3.65,
 '5501': 3.75}

In [20]:
# Predict Sales Qty and Compare to Actual

import requests
import json

url = 'http://localhost:5000/predict/'

payload = {'prices': a_input, 'project_id': 1}

headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)
qty = r.json()['qty_estimates']
qty_outp = {}
for key in qty.keys():
    qty_outp[key] = round(qty[key], 0)
actual = a[[
    col for col in sales_data_wide_clean.columns if col.startswith('Qty_')
]].iloc[0].to_dict()
pd.DataFrame([qty_outp, actual], index=['Estimated', 'Actual']).transpose()

,Estimated,Actual
Qty_102,122.0,91.0
Qty_1203,227.0,222.0
Qty_197,79.0,74.0
Qty_2259,94.0,63.0
Qty_31,67.0,78.0
Qty_3767,269.0,267.0
Qty_3818,69.0,82.0
Qty_4270,37.0,67.0
Qty_5073,189.0,232.0
Qty_5501,558.0,453.0
